Links
https://op.europa.eu/en/web/eu-vocabularies/e-procurement/tedschemas

https://ec.europa.eu/docsroom/documents/27821/attachments/1/translations/en/renditions/native page 22

link to files https://1drv.ms/u/s!Ag_DGaic1jWMlY1HKM2GKxf_o76L3w?e=CptwgP

In [ ]:
pip install xmltodict

In [ ]:
pip install lxml

In [ ]:
from pyspark import pandas as ps
import zipfile
#import xmltodict
import re
import numpy as np
import os
import json
import glob
import shutil
#import lxml.etree as ET
#import pandas as pd
from pyspark.sql import *
from copy import copy

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, concat_ws, lit, col, trim, expr
from pyspark.sql.types import StructType, StructField, StringType,IntegerType

os.environ["PYARROW_IGNORE_TIMEZONE"]="1"

def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster('local[*]')
    conf \
      .set('spark.driver.memory', '64g')\
      .set("fs.s3a.access.key", "minio") \
      .set("fs.s3a.secret.key", "minio123") \
      .set("fs.s3a.endpoint", "http://192.168.1.127:9000") \
      .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
      .set("spark.hadoop.fs.s3a.path.style.access", "true") \
      .set("spark.sql.repl.eagerEval.enabled", "True") \
      .set("spark.sql.adaptive.enabled", "True") \
      .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
      .set("spark.sql.repl.eagerEval.maxNumRows", "10000") \
      .set("spark.setLogLevel", "error")
    
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()

spark = get_spark_session("Falk", SparkConf())

In [ ]:
spark.stop()

In [ ]:
def xml_to_json(dir_path):
    path = dir_path 
    for filename in os.listdir(path):
        if not filename.endswith('.xml'):
            continue

        fullname = os.path.join(path, filename)

        with open(fullname, 'r') as f:
            xmlString = f.read()
            
        
        jsonString = xmltodict.parse(xmlString, process_namespaces=True, attr_prefix='', cdata_key="text")
        #jsonString = json.dumps(xmltodict.parse(xmlString, process_namespaces=True, attr_prefix='', cdata_key="text"),ensure_ascii=False)
    #jsonString = (xmltodict.parse(xmlString, process_namespaces=True))
    #data = json.load(jsonString)
    #jsondata = json.load(jsonString, object_hook=lambda d: {int(k): [int(i) for i in v] if isinstance(v, list) else v for k, v in d.items()})
    #jsondata2 = json.dumps(jsondata)
    #test.import_bulk(data)
    #test1 = json.load(jsonString, parse_float=float)
    #test.insert(jsonString)
    #print(jsonString)

        #with open(fullname[:-4] + ".json", 'w') as f:
         #   f.write(jsonString)
            
                                     
        with open(fullname[:-4] + ".json", 'w', encoding ='utf8') as json_file:
                                     json.dump(jsonString, json_file, ensure_ascii = False)
                                     

In [ ]:
def _decode(o):
    # Cassting sting to int or float
    
    
    if isinstance(o, str):
        try:
            return int(o)
        except ValueError:
            try:
                return float(o)
            except ValueError:
                return o

    elif isinstance(o, dict):
        return {k: _decode(v) for k, v in o.items()}
    elif isinstance(o, list):
        return [_decode(v) for v in o]
    else:
        return o       
        

In [ ]:
def str_to_numb(dir_path):
    path = dir_path
    for filename in os.listdir(path):
        if not filename.endswith('.json'):
            continue


        fullname = os.path.join(path, filename)
        with open(fullname) as json_file:
            jsonstr = json.load(json_file)
    
        #with open(fullname, 'r') as f:
         #   jsonstr = f.read()

        #json_sting = json.loads(jsonstr)
        for item in _decode(jsonstr):
            with open(fullname[:-4] + "_decode.json", 'w', encoding ='utf8') as f:
                json.dump(jsonstr, f, ensure_ascii = False)
                


In [ ]:
def recursive_iter(obj):
    if isinstance(obj, dict):
        for item in obj.values():
            if "P" in obj and isinstance(obj["P"], list):
                obj["P"] = " ".join([str(e) for e in obj["P"]])
            else:
                yield from recursive_iter(item)
    elif any(isinstance(obj, t) for t in (list, tuple)):
        for item in obj:
            yield from recursive_iter(item)
    else:
        yield obj
        
        

In [ ]:
def json_strip_list2(dir_path):
    path = dir_path 

    for filename in os.listdir(path):
        if not filename.endswith('.json'):
            continue


        fullname = os.path.join(path, filename)
        with open(fullname) as json_file:
            jsonstr = json.load(json_file, object_hook= _decode)
    
        #with open(fullname, 'r') as f:
         #   jsonstr = f.read()

        #json_sting = json.loads(jsonstr)
        for item in recursive_iter(jsonstr):
            with open(fullname[:-4] + "_stript_list.json", 'w', encoding ='utf8') as f:
                json.dump(jsonstr, f, ensure_ascii = False)
            #json_str2 = json.dump(jsonstr)
        
        
        #with open(fullname[:-4] + "_stript_list.json", 'w') as f:
         #   json.dump(json_str2, f)
        
        
        
        #with open(fullname[:-4] + "_stript_list.json", 'w') as f:
         #   f.write(json_str2)   
         
         
       

In [ ]:
# We take a dataframe and return a new one with required changes
def cleanDataFrame(df: DataFrame) -> DataFrame:
    # Returns a new sanitized field name (this function can be anything really)
    def sanitizeFieldName(s: str) -> str:
        return s.replace("-", "_").replace("&", "_").replace("\"", "_")\
            .replace("[", "_").replace("]", "_").replace(".", "_")

    # We call this on all fields to create a copy and to perform any changes we might
    # want to do to the field.
    def sanitizeField(field: StructField) -> StructField:
        field = copy(field)
        field.name = sanitizeFieldName(field.name)
        # We recursively call cleanSchema on all types
        field.dataType = cleanSchema(field.dataType)
        return field

    def cleanSchema(dataType: [DataType]) -> [DateType]:
        dataType = copy(dataType)
        # If the type is a StructType we need to recurse otherwise we can return since
        # we've reached the leaf node
        if isinstance(dataType, StructType):
            # We call our sanitizer for all top level fields
            dataType.fields = [sanitizeField(f) for f in dataType.fields]
        elif isinstance(dataType, ArrayType):
            dataType.elementType = cleanSchema(dataType.elementType)
        return dataType

    # Now since we have the new schema we can create a new DataFrame by using the old Frame's RDD as data and the new schema as the schema for the data
    return spark.createDataFrame(df.rdd, cleanSchema(df.schema))     
    
    
    

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


def flatten_test(df, sep="_"):
    """Returns a flattened dataframe.
        .. versionadded:: x.X.X
        
        Parameters
        ----------
        sep : str
            Delimiter for flatted columns. Default `_`
        
        Notes
        -----
        Don`t use `.` as `sep`
        It won't work on nested data frames with more than one level.
        And you will have to use `columns.name`. 
        
        Flattening Map Types will have to find every key in the column. 
        This can be slow.
        
        Examples
        --------

        data_mixed = [
            {
                "state": "Florida",
                "shortname": "FL",
                "info": {"governor": "Rick Scott"},
                "counties": [
                    {"name": "Dade", "population": 12345},
                    {"name": "Broward", "population": 40000},
                    {"name": "Palm Beach", "population": 60000},
                ],
            },
            {
                "state": "Ohio",
                "shortname": "OH",
                "info": {"governor": "John Kasich"},
                "counties": [
                    {"name": "Summit", "population": 1234},
                    {"name": "Cuyahoga", "population": 1337},
                ],
            },
        ]

        data_mixed = spark.createDataFrame(data=data_mixed)

        data_mixed.printSchema()

        root
        |-- counties: array (nullable = true)
        |    |-- element: map (containsNull = true)
        |    |    |-- key: string
        |    |    |-- value: string (valueContainsNull = true)
        |-- info: map (nullable = true)
        |    |-- key: string
        |    |-- value: string (valueContainsNull = true)
        |-- shortname: string (nullable = true)
        |-- state: string (nullable = true)
        
        
        data_mixed_flat = flatten_test(df, sep=":")
        data_mixed_flat.printSchema()
        root
        |-- shortname: string (nullable = true)
        |-- state: string (nullable = true)
        |-- counties:name: string (nullable = true)
        |-- counties:population: string (nullable = true)
        |-- info:governor: string (nullable = true)
        

        
        
        data = [
            {
                "id": 1,
                "name": "Cole Volk",
                "fitness": {"height": 130, "weight": 60},
            },
            {"name": "Mark Reg", "fitness": {"height": 130, "weight": 60}},
            {
                "id": 2,
                "name": "Faye Raker",
                "fitness": {"height": 130, "weight": 60},
            },
        ]


        df = spark.createDataFrame(data=data)

        df.printSchema()

        root
        |-- fitness: map (nullable = true)
        |    |-- key: string
        |    |-- value: long (valueContainsNull = true)
        |-- id: long (nullable = true)
        |-- name: string (nullable = true)
        
        df_flat = flatten_test(df, sep=":")

        df_flat.printSchema()

        root
        |-- id: long (nullable = true)
        |-- name: string (nullable = true)
        |-- fitness:height: long (nullable = true)
        |-- fitness:weight: long (nullable = true)
        
        data_struct = [
                (("James",None,"Smith"),"OH","M"),
                (("Anna","Rose",""),"NY","F"),
                (("Julia","","Williams"),"OH","F"),
                (("Maria","Anne","Jones"),"NY","M"),
                (("Jen","Mary","Brown"),"NY","M"),
                (("Mike","Mary","Williams"),"OH","M")
                ]

                
        schema = StructType([
            StructField('name', StructType([
                StructField('firstname', StringType(), True),
                StructField('middlename', StringType(), True),
                StructField('lastname', StringType(), True)
                ])),
            StructField('state', StringType(), True),
            StructField('gender', StringType(), True)
            ])

        df_struct = spark.createDataFrame(data = data_struct, schema = schema)

        df_struct.printSchema()

        root
        |-- name: struct (nullable = true)
        |    |-- firstname: string (nullable = true)
        |    |-- middlename: string (nullable = true)
        |    |-- lastname: string (nullable = true)
        |-- state: string (nullable = true)
        |-- gender: string (nullable = true)
        
        df_struct_flat = flatten_test(df_struct, sep=":")

        df_struct_flat.printSchema()

        root
        |-- state: string (nullable = true)
        |-- gender: string (nullable = true)
        |-- name:firstname: string (nullable = true)
        |-- name:middlename: string (nullable = true)
        |-- name:lastname: string (nullable = true)
        """
    # compute Complex Fields (Arrays, Structs and Maptypes) in Schema   
    complex_fields = dict([(field.name, field.dataType)
                            for field in df.schema.fields
                            if type(field.dataType) == ArrayType 
                            or type(field.dataType) == StructType
                            or type(field.dataType) == MapType])
    
    while len(complex_fields) !=0:
        col_name = list(complex_fields.keys())[0]
        #print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))
    
        # if StructType then convert all sub element to columns.
        # i.e. flatten structs
        if (type(complex_fields[col_name]) == StructType):
            expanded = [col(col_name + '.' + k).alias(col_name + sep + k) 
            for k in [n.name for n in complex_fields[col_name]]]
            df = df.select("*", *expanded).drop(col_name)
    
        # if ArrayType then add the Array Elements as Rows using the explode function
        # i.e. explode Arrays
        elif (type(complex_fields[col_name]) == ArrayType):
            df = df.withColumn(col_name, explode_outer(col_name))
        
        # if MapType then convert all sub element to columns.
        # i.e. flatten
        elif (type(complex_fields[col_name]) == MapType):
            keys_df = df.select(explode_outer(map_keys(col(col_name)))).distinct()
            keys = list(map(lambda row: row[0], keys_df.collect()))
            key_cols = list(map(lambda f: col(col_name).getItem(f)
            .alias(str(col_name + sep + f)), keys))
            drop_column_list = [col_name]
            df = df.select([col_name for col_name in df.columns 
            if col_name not in drop_column_list] + key_cols)
    
        # recompute remaining Complex Fields in Schema       
        complex_fields = dict([(field.name, field.dataType)
                            for field in df.schema.fields
                            if type(field.dataType) == ArrayType
                            or type(field.dataType) == StructType
                            or type(field.dataType) == MapType])

    return df

In [ ]:
def json_to_norm(dir_path, path_to_save):
    path = dir_path 

    for filename in os.listdir(path):
        if not filename.endswith('._stript_list.json'):
            continue


        fullname = os.path.join(path, filename)
        with open(fullname) as json_file:
            jsonstr = json.load(json_file)
    
        #with open(fullname, 'r') as f:
         #   jsonstr = f.read()

        #json_sting = json.loads(jsonstr)
        #for item in jsonstr():
        df = spark.read.json(fullname)
        df = cleanDataFrame(df)
        df = flatten_test(df, sep=":")
        df.write.mode('append').json(path_to_save)
        #df3.write.mode('append').json('/home/jovyan/notebooks/falk/F02.json')
        #df3.coalesce(1).write.mode('append').json('/home/jovyan/notebooks/falk/F01.json')
        
        
        
      

In [ ]:
def json_to_norm_with_null(dir_path, path_to_save):
    path = dir_path 

    for filename in os.listdir(path):
        if not filename.endswith('._stript_list.json'):
            continue


        fullname = os.path.join(path, filename)
        with open(fullname) as json_file:
            jsonstr = json.load(json_file)
    
        #with open(fullname, 'r') as f:
         #   jsonstr = f.read()

        #json_sting = json.loads(jsonstr)
        #for item in jsonstr():
        df = spark.read.json(fullname)
        df = cleanDataFrame(df)
        df = flatten_test(df, sep=":")
        df.write.mode('append').option("ignoreNullFields", "false").json(path_to_save)
        #df3.write.mode('append').json('/home/jovyan/notebooks/falk/F02.json')
        #df3.coalesce(1).write.mode('append').json('/home/jovyan/notebooks/falk/F01.json')

In [ ]:
#%%
#Then, I define the working directory:

#https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5

working_directory = '/home/jovyan/notebooks/falk/data/'
os.chdir(working_directory)

#After that you can use a combination of the os and zipfile to get where you want:
#run this 4 times! if you get folders zip files in folders ain't unzipt
for file in os.listdir(working_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile, extract it
        with zipfile.ZipFile(file) as item: # treat the file as a zip
           item.extractall()  # extract it in the working directory        

In [ ]:
#%%
#Then, I define the working directory:

#https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5

working_directory = '/home/jovyan/notebooks/falk/data/'
os.chdir(working_directory)

#After that you can use a combination of the os and zipfile to get where you want:
#run this 4 times! if you get folders zip files in folders ain't unzipt
for file in os.listdir(working_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile, extract it
        with zipfile.ZipFile(file) as item: # treat the file as a zip
           item.extractall()  # extract it in the working directory  

In [ ]:
#%%
#Then, I define the working directory:

#https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5

working_directory = '/home/jovyan/notebooks/falk/data/'
os.chdir(working_directory)

#After that you can use a combination of the os and zipfile to get where you want:
#run this 4 times! if you get folders zip files in folders ain't unzipt
for file in os.listdir(working_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile, extract it
        with zipfile.ZipFile(file) as item: # treat the file as a zip
           item.extractall()  # extract it in the working directory  

In [ ]:
#%%
#Then, I define the working directory:

#https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5

working_directory = '/home/jovyan/notebooks/falk/data/'
os.chdir(working_directory)

#After that you can use a combination of the os and zipfile to get where you want:
#run this 4 times! if you get folders zip files in folders ain't unzipt
for file in os.listdir(working_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile, extract it
        with zipfile.ZipFile(file) as item: # treat the file as a zip
           item.extractall()  # extract it in the working directory  

In [ ]:
ls -l | wc -l

In [ ]:
pwd

In [ ]:
mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/doffin-2018-11/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/doffin-2018-12/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-2018/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201802/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201803/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201804/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201805/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201806/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201807/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201808/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201809/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-201810/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-202008/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-202009/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/
[bjorn@amd7g data]$ mv /home/bjorn/jupyter-spark-local/notebooks/falk/data/Doffin-202010/*.* /home/bjorn/jupyter-spark-local/notebooks/falk/data/



[bjorn@amd7g data]$ rm -R doffin-2018-11
[bjorn@amd7g data]$ rm -R doffin-2018-12
[bjorn@amd7g data]$ rm -R Doffin-201802
[bjorn@amd7g data]$ rm -R Doffin-201803
[bjorn@amd7g data]$ rm -R Doffin-201804
[bjorn@amd7g data]$ rm -R Doffin-201805
[bjorn@amd7g data]$ rm -R Doffin-201806
[bjorn@amd7g data]$ rm -R Doffin-201807
[bjorn@amd7g data]$ rm -R Doffin-201808
[bjorn@amd7g data]$ rm -R Doffin-201809
[bjorn@amd7g data]$ rm -R Doffin-201810
[bjorn@amd7g data]$ rm -R Doffin-202008
[bjorn@amd7g data]$ rm -R Doffin-202009
[bjorn@amd7g data]$ rm -R Doffin-202010

In [ ]:
#%%
#Then, I define the working directory:

#https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5

working_directory = '/home/jovyan/notebooks/falk/data/'
os.chdir(working_directory)

#After that you can use a combination of the os and zipfile to get where you want:
#run this 4 times! if you get folders zip files in folders ain't unzipt
for file in os.listdir(working_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile, extract it
        with zipfile.ZipFile(file) as item: # treat the file as a zip
           item.extractall()  # extract it in the working directory  

In [ ]:
ls -l | wc -l

In [ ]:
#Files with UTF errors
!rm 2017-720627.xml
!rm 2017-046963.xml
!rm 2017-879966.xml
!rm 2017-862307.xml
!rm 2017-181110.xml
!rm 2017-681348.xml

In [ ]:
cp UTF82017-* falk/data/

In [ ]:
ls -l | wc -l


In [ ]:
pwd

## Note source data must be in another folder then xml sorting  https://serveanswer.com/questions/problem-on-error-session-line-number-was-not-unique-in-database-history-logging-moved-to-new-session-121

In [ ]:
!mkdir /home/jovyan/notebooks/falk/data/form_version

In [ ]:
!mkdir /home/jovyan/notebooks/falk/data/form_version/F01
!mkdir /home/jovyan/notebooks/falk/data/form_version/F02
!mkdir /home/jovyan/notebooks/falk/data/form_version/F03
!mkdir /home/jovyan/notebooks/falk/data/form_version/F04
!mkdir /home/jovyan/notebooks/falk/data/form_version/F05
!mkdir /home/jovyan/notebooks/falk/data/form_version/F06
!mkdir /home/jovyan/notebooks/falk/data/form_version/F07
!mkdir /home/jovyan/notebooks/falk/data/form_version/F08
!mkdir /home/jovyan/notebooks/falk/data/form_version/F09
!mkdir /home/jovyan/notebooks/falk/data/form_version/F12
!mkdir /home/jovyan/notebooks/falk/data/form_version/F13
!mkdir /home/jovyan/notebooks/falk/data/form_version/F14
!mkdir /home/jovyan/notebooks/falk/data/form_version/F15
!mkdir /home/jovyan/notebooks/falk/data/form_version/F20
!mkdir /home/jovyan/notebooks/falk/data/form_version/F21
!mkdir /home/jovyan/notebooks/falk/data/form_version/F22
!mkdir /home/jovyan/notebooks/falk/data/form_version/F23
!mkdir /home/jovyan/notebooks/falk/data/form_version/F24
!mkdir /home/jovyan/notebooks/falk/data/form_version/F25
!mkdir /home/jovyan/notebooks/falk/data/form_version/F52
!mkdir /home/jovyan/notebooks/falk/data/form_version/F65
!mkdir /home/jovyan/notebooks/falk/data/form_version/F102

!mkdir /home/jovyan/notebooks/falk/data/form_version/1
!mkdir /home/jovyan/notebooks/falk/data/form_version/2
!mkdir /home/jovyan/notebooks/falk/data/form_version/3
!mkdir /home/jovyan/notebooks/falk/data/form_version/4
!mkdir /home/jovyan/notebooks/falk/data/form_version/5
!mkdir /home/jovyan/notebooks/falk/data/form_version/6
!mkdir /home/jovyan/notebooks/falk/data/form_version/7
!mkdir /home/jovyan/notebooks/falk/data/form_version/8
!mkdir /home/jovyan/notebooks/falk/data/form_version/9
!mkdir /home/jovyan/notebooks/falk/data/form_version/10
!mkdir /home/jovyan/notebooks/falk/data/form_version/11
!mkdir /home/jovyan/notebooks/falk/data/form_version/12
!mkdir /home/jovyan/notebooks/falk/data/form_version/13
!mkdir /home/jovyan/notebooks/falk/data/form_version/14
!mkdir /home/jovyan/notebooks/falk/data/form_version/15
!mkdir /home/jovyan/notebooks/falk/data/form_version/16
!mkdir /home/jovyan/notebooks/falk/data/form_version/17
!mkdir /home/jovyan/notebooks/falk/data/form_version/18
!mkdir /home/jovyan/notebooks/falk/data/form_version/101
!mkdir /home/jovyan/notebooks/falk/data/form_version/102
!mkdir /home/jovyan/notebooks/falk/data/form_version/103

!mkdir /home/jovyan/notebooks/falk/data/form_version/v_1_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_2_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_3_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_4_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_5_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_6_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_7_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_8_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_9_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_10_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_11_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_12_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_13_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_14_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_51_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_52_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_53_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_61_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_62_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_63_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_64_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_68_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_69_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_70_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_72_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_81_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_82_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_83_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_84_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_85_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_86_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_87_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_88_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_89_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_90_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_91_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_92_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/v_97_before_2014

!mkdir /home/jovyan/notebooks/falk/data/form_version/v_TED_ESENDERS_before_2014
!mkdir /home/jovyan/notebooks/falk/data/form_version/other

In [ ]:
source_dir = "/home/jovyan/notebooks/falk/data2/"
dest_dir_F01 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_F02 = "/home/jovyan/notebooks/falk/data/form_version/F02"
dest_dir_F03 = "/home/jovyan/notebooks/falk/data/form_version/F03"
dest_dir_F04 = "/home/jovyan/notebooks/falk/data/form_version/F04"
dest_dir_F05 = "/home/jovyan/notebooks/falk/data/form_version/F05"
dest_dir_F06 = "/home/jovyan/notebooks/falk/data/form_version/F06"
dest_dir_F07 = "/home/jovyan/notebooks/falk/data/form_version/F07"
dest_dir_F08 = "/home/jovyan/notebooks/falk/data/form_version/F08"
dest_dir_F09 = "/home/jovyan/notebooks/falk/data/form_version/F09"
dest_dir_F12 = "/home/jovyan/notebooks/falk/data/form_version/F12"
dest_dir_F13 = "/home/jovyan/notebooks/falk/data/form_version/F13"
dest_dir_F14 = "/home/jovyan/notebooks/falk/data/form_version/F14"
dest_dir_F15 = "/home/jovyan/notebooks/falk/data/form_version/F15"
dest_dir_F20 = "/home/jovyan/notebooks/falk/data/form_version/F20"
dest_dir_F21 = "/home/jovyan/notebooks/falk/data/form_version/F21"
dest_dir_F22 = "/home/jovyan/notebooks/falk/data/form_version/F22"
dest_dir_F23 = "/home/jovyan/notebooks/falk/data/form_version/F23"
dest_dir_F24 = "/home/jovyan/notebooks/falk/data/form_version/F24"
dest_dir_F25 = "/home/jovyan/notebooks/falk/data/form_version/F25"
dest_dir_F52 = "/home/jovyan/notebooks/falk/data/form_version/F52"
dest_dir_F65 = "/home/jovyan/notebooks/falk/data/form_version/F65"
dest_dir_F102 = "/home/jovyan/notebooks/falk/data/form_version/F102"

dest_dir_1 = "/home/jovyan/notebooks/falk/data/form_version/1"
dest_dir_2 = "/home/jovyan/notebooks/falk/data/form_version/2"
dest_dir_3 = "/home/jovyan/notebooks/falk/data/form_version/3"
dest_dir_4 = "/home/jovyan/notebooks/falk/data/form_version/4"
dest_dir_5 = "/home/jovyan/notebooks/falk/data/form_version/5"
dest_dir_6 = "/home/jovyan/notebooks/falk/data/form_version/6"
dest_dir_7 = "/home/jovyan/notebooks/falk/data/form_version/7"
dest_dir_8 = "/home/jovyan/notebooks/falk/data/form_version/8"
dest_dir_9 = "/home/jovyan/notebooks/falk/data/form_version/9"
dest_dir_10 = "/home/jovyan/notebooks/falk/data/form_version/10"
dest_dir_11 = "/home/jovyan/notebooks/falk/data/form_version/11"
dest_dir_12 = "/home/jovyan/notebooks/falk/data/form_version/12"
dest_dir_13 = "/home/jovyan/notebooks/falk/data/form_version/13"
dest_dir_14 = "/home/jovyan/notebooks/falk/data/form_version/14"
dest_dir_15 = "/home/jovyan/notebooks/falk/data/form_version/15"
dest_dir_16 = "/home/jovyan/notebooks/falk/data/form_version/16"
dest_dir_17 = "/home/jovyan/notebooks/falk/data/form_version/17"
dest_dir_18 = "/home/jovyan/notebooks/falk/data/form_version/18"
dest_dir_101 = "/home/jovyan/notebooks/falk/data/form_version/101"
dest_dir_102 = "/home/jovyan/notebooks/falk/data/form_version/102"
dest_dir_103 = "/home/jovyan/notebooks/falk/data/form_version/103"
dest_dir_R2_0_8_S02 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_R2_0_9_S01 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_V2_0_0 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_other = "/home/jovyan/notebooks/falk/data/form_version/other"


dest_dir_v_1_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_1_before_2014"
dest_dir_v_2_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_2_before_2014"
dest_dir_v_3_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_3_before_2014"
dest_dir_v_4_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_4_before_2014"
dest_dir_v_5_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_5_before_2014"
dest_dir_v_6_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_6_before_2014"
dest_dir_v_7_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_7_before_2014"
dest_dir_v_8_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_8_before_2014"
dest_dir_v_9_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_9_before_2014"
dest_dir_v_10_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_10_before_2014"
dest_dir_v_11_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_11_before_2014"
dest_dir_v_12_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_12_before_2014"
dest_dir_v_13_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_13_before_2014"
dest_dir_v_14_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_14_before_2014"
dest_dir_v_51_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_51_before_2014"
dest_dir_v_52_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_52_before_2014"
dest_dir_v_53_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_53_before_2014"
dest_dir_v_61_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_61_before_2014"
dest_dir_v_62_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_62_before_2014"
dest_dir_v_63_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_63_before_2014"
dest_dir_v_64_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_64_before_2014"
dest_dir_v_68_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_68_before_2014"
dest_dir_v_69_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_69_before_2014"
dest_dir_v_70_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_70_before_2014"
dest_dir_v_72_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_72_before_2014"
dest_dir_v_81_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_81_before_2014"
dest_dir_v_82_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_82_before_2014"
dest_dir_v_83_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_83_before_2014"
dest_dir_v_84_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_84_before_2014"
dest_dir_v_85_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_85_before_2014"
dest_dir_v_86_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_86_before_2014"
dest_dir_v_87_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_87_before_2014"
dest_dir_v_88_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_88_before_2014"
dest_dir_v_89_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_89_before_2014"
dest_dir_v_90_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_90_before_2014"
dest_dir_v_91_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_91_before_2014"
dest_dir_v_92_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_92_before_2014"
dest_dir_v_97_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_97_before_2014"

dest_dir_v_TED_ESENDERS_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_TED_ESENDERS_before_2014"




for fn in os.listdir(source_dir):
    tree = ET.parse(os.path.join(source_dir,fn))
    root = tree.getroot()
    #root_tag = root.tag
    #print(root_tag)
    #print(root[0][0].attrib)
    #root1 = tree.getroot([DOFFIN_ESENDERS])
    #print(root1)
    #test = root.get()
    v = root[0][0].get('FORM')
    #v_301 = root[0][1].get('FORM')
    v_root_tag = root.tag
    v_root_before_2014 = root.get('FORM')
    #print(v)
    #print(root)
    
    if v == "1":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_1, fn))
    elif v == "2":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_2, fn))

    elif v == "3":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_3, fn))
    elif v == "4":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_4, fn))
    elif v == "5":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_5, fn))
    elif v == "6":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_6, fn))
    elif v == "7":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_7, fn))
    elif v == "8":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_8, fn))
    elif v == "9":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_9, fn))
    elif v == "10":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_10, fn))
    elif v == "11":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_11, fn))
    elif v == "12":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_12, fn))
    elif v == "13":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_13, fn))
    elif v == "14":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_14, fn))
    elif v == "15":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_15, fn))
    elif v == "16":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_16, fn))
    elif v == "17":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_17, fn))
    elif v == "18":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_18, fn))
    elif v == "101":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_101, fn))
    elif v == "102":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_102, fn))
    elif v == "103":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_103, fn))
    elif v == "F01":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F01, fn))
    elif v == "F02":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F02, fn))
    elif v == "F03":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F03, fn))
    elif v == "F04":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F04, fn))
    elif v == "F05":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F05, fn))
    elif v == "F06":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F06, fn))
    elif v == "F07":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F07, fn))
    elif v == "F08":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F08, fn))
    elif v == "F09":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F09, fn))
    elif v == "F12":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F12, fn))
    elif v == "F13":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F13, fn))
    elif v == "F14":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F14, fn))
    elif v == "F15":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F15, fn))
    elif v == "F20":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F20, fn))
    elif v == "F21":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F21, fn))
    elif v == "F22":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F22, fn))
    elif v == "F23":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F23, fn))
    elif v == "F24":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F24, fn))
    elif v == "F25":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F25, fn))
    elif v == "F52":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F52, fn))
    elif v == "F65":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F65, fn))
    elif v == "F102":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F102, fn))
    #elif v_301 == "F01":
     #   shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F01, fn))
    #elif v_301 == "F02":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F02, fn))
    #elif v_301 == "F03":
     #   shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F03, fn))
    #elif v_301 == "F04":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F04, fn))
    #elif v_301 == "F05":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F05, fn))
    #elif v_301 == "F06":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F06, fn))
    #elif v_301 == "F07":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F07, fn))
    #elif v_301 == "F08":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F08, fn))
    #elif v_301 == "F09":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F09, fn))
    #elif v_301 == "F12":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F12, fn))
    #elif v_301 == "F13":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F13, fn))
    #elif v_301 == "F14":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F14, fn))
    #elif v_301 == "F15":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F15, fn))
    #elif v_301 == "F20":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F20, fn))
    #elif v_301 == "F21":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F21, fn))
    #elif v_301 == "F22":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F22, fn))
    #elif v_301 == "F23":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F23, fn))
    #elif v_301 == "F24":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F24, fn))
    #elif v_301 == "F25":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F25, fn))
    #elif v_301 == "F52":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F52, fn))
    #elif v_301 == "F65":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F65, fn))
    #elif v_301 == "F102":
    #    shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F102, fn))
    elif v_root_before_2014 == "1":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_1_before_2014, fn))
    elif v_root_before_2014 == "2":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_2_before_2014, fn))
    elif v_root_before_2014 == "3":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_3_before_2014, fn))
    elif v_root_before_2014 == "4":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_4_before_2014, fn))
    elif v_root_before_2014 == "5":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_5_before_2014, fn))
    elif v_root_before_2014 == "6":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_6_before_2014, fn))
    elif v_root_before_2014 == "7":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_7_before_2014, fn))
    elif v_root_before_2014 == "8":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_8_before_2014, fn))
    elif v_root_before_2014 == "9":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_9_before_2014, fn))
    elif v_root_before_2014 == "10":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_10_before_2014, fn))
    elif v_root_before_2014 == "11":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_10_before_2014, fn))
    elif v_root_before_2014 == "12":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_12_before_2014, fn))
    elif v_root_before_2014 == "13":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_13_before_2014, fn))
    elif v_root_before_2014 == "14":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_14_before_2014, fn))
    elif v_root_before_2014 == "51":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_51_before_2014, fn))
    elif v_root_before_2014 == "52":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_52_before_2014, fn))
    elif v_root_before_2014 == "53":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_53_before_2014, fn))
    elif v_root_before_2014 == "61":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_61_before_2014, fn))
    elif v_root_before_2014 == "62":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_62_before_2014, fn))
    elif v_root_before_2014 == "63":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_63_before_2014, fn))
    elif v_root_before_2014 == "64":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_64_before_2014, fn))
    elif v_root_before_2014 == "68":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_68_before_2014, fn))
    elif v_root_before_2014 == "69":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_69_before_2014, fn))
    elif v_root_before_2014 == "70":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_70_before_2014, fn))
    elif v_root_before_2014 == "72":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_72_before_2014, fn))
    elif v_root_before_2014 == "81":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_81_before_2014, fn))
    elif v_root_before_2014 == "82":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_82_before_2014, fn))
    elif v_root_before_2014 == "83":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_83_before_2014, fn))
    elif v_root_before_2014 == "84":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_84_before_2014, fn))
    elif v_root_before_2014 == "85":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_85_before_2014, fn))
    elif v_root_before_2014 == "86":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_86_before_2014, fn))
    elif v_root_before_2014 == "87":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_87_before_2014, fn))
    elif v_root_before_2014 == "88":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_88_before_2014, fn))
    elif v_root_before_2014 == "89":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_89_before_2014, fn))
    elif v_root_before_2014 == "90":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_90_before_2014, fn))
    elif v_root_before_2014 == "91":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_91_before_2014, fn))
    elif v_root_before_2014 == "92":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_92_before_2014, fn))
    elif v_root_before_2014 == "97":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_97_before_2014, fn))
    elif v_root_tag == "TED_ESENDERS":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_TED_ESENDERS_before_2014, fn))
    
    else:
        #shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_other, fn))
        pass

In [ ]:
source_dir = "/home/jovyan/notebooks/falk/data2/"
dest_dir_F01 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_F02 = "/home/jovyan/notebooks/falk/data/form_version/F02"
dest_dir_F03 = "/home/jovyan/notebooks/falk/data/form_version/F03"
dest_dir_F04 = "/home/jovyan/notebooks/falk/data/form_version/F04"
dest_dir_F05 = "/home/jovyan/notebooks/falk/data/form_version/F05"
dest_dir_F06 = "/home/jovyan/notebooks/falk/data/form_version/F06"
dest_dir_F07 = "/home/jovyan/notebooks/falk/data/form_version/F07"
dest_dir_F08 = "/home/jovyan/notebooks/falk/data/form_version/F08"
dest_dir_F09 = "/home/jovyan/notebooks/falk/data/form_version/F09"
dest_dir_F12 = "/home/jovyan/notebooks/falk/data/form_version/F12"
dest_dir_F13 = "/home/jovyan/notebooks/falk/data/form_version/F13"
dest_dir_F14 = "/home/jovyan/notebooks/falk/data/form_version/F14"
dest_dir_F15 = "/home/jovyan/notebooks/falk/data/form_version/F15"
dest_dir_F20 = "/home/jovyan/notebooks/falk/data/form_version/F20"
dest_dir_F21 = "/home/jovyan/notebooks/falk/data/form_version/F21"
dest_dir_F22 = "/home/jovyan/notebooks/falk/data/form_version/F22"
dest_dir_F23 = "/home/jovyan/notebooks/falk/data/form_version/F23"
dest_dir_F24 = "/home/jovyan/notebooks/falk/data/form_version/F24"
dest_dir_F25 = "/home/jovyan/notebooks/falk/data/form_version/F25"
dest_dir_F52 = "/home/jovyan/notebooks/falk/data/form_version/F52"
dest_dir_F65 = "/home/jovyan/notebooks/falk/data/form_version/F65"
dest_dir_F102 = "/home/jovyan/notebooks/falk/data/form_version/F102"

dest_dir_1 = "/home/jovyan/notebooks/falk/data/form_version/1"
dest_dir_2 = "/home/jovyan/notebooks/falk/data/form_version/2"
dest_dir_3 = "/home/jovyan/notebooks/falk/data/form_version/3"
dest_dir_4 = "/home/jovyan/notebooks/falk/data/form_version/4"
dest_dir_5 = "/home/jovyan/notebooks/falk/data/form_version/5"
dest_dir_6 = "/home/jovyan/notebooks/falk/data/form_version/6"
dest_dir_7 = "/home/jovyan/notebooks/falk/data/form_version/7"
dest_dir_8 = "/home/jovyan/notebooks/falk/data/form_version/8"
dest_dir_9 = "/home/jovyan/notebooks/falk/data/form_version/9"
dest_dir_10 = "/home/jovyan/notebooks/falk/data/form_version/10"
dest_dir_11 = "/home/jovyan/notebooks/falk/data/form_version/11"
dest_dir_12 = "/home/jovyan/notebooks/falk/data/form_version/12"
dest_dir_13 = "/home/jovyan/notebooks/falk/data/form_version/13"
dest_dir_14 = "/home/jovyan/notebooks/falk/data/form_version/14"
dest_dir_15 = "/home/jovyan/notebooks/falk/data/form_version/15"
dest_dir_16 = "/home/jovyan/notebooks/falk/data/form_version/16"
dest_dir_17 = "/home/jovyan/notebooks/falk/data/form_version/17"
dest_dir_18 = "/home/jovyan/notebooks/falk/data/form_version/18"
dest_dir_101 = "/home/jovyan/notebooks/falk/data/form_version/101"
dest_dir_102 = "/home/jovyan/notebooks/falk/data/form_version/102"
dest_dir_103 = "/home/jovyan/notebooks/falk/data/form_version/103"
dest_dir_R2_0_8_S02 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_R2_0_9_S01 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_V2_0_0 = "/home/jovyan/notebooks/falk/data/form_version/F01"
dest_dir_other = "/home/jovyan/notebooks/falk/data/form_version/other"


dest_dir_v_1_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_1_before_2014"
dest_dir_v_2_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_2_before_2014"
dest_dir_v_3_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_3_before_2014"
dest_dir_v_4_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_4_before_2014"
dest_dir_v_5_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_5_before_2014"
dest_dir_v_6_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_6_before_2014"
dest_dir_v_7_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_7_before_2014"
dest_dir_v_8_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_8_before_2014"
dest_dir_v_9_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_9_before_2014"
dest_dir_v_10_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_10_before_2014"
dest_dir_v_11_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_11_before_2014"
dest_dir_v_12_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_12_before_2014"
dest_dir_v_13_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_13_before_2014"
dest_dir_v_14_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_14_before_2014"
dest_dir_v_51_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_51_before_2014"
dest_dir_v_52_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_52_before_2014"
dest_dir_v_53_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_53_before_2014"
dest_dir_v_61_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_61_before_2014"
dest_dir_v_62_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_62_before_2014"
dest_dir_v_63_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_63_before_2014"
dest_dir_v_64_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_64_before_2014"
dest_dir_v_68_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_68_before_2014"
dest_dir_v_69_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_69_before_2014"
dest_dir_v_70_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_70_before_2014"
dest_dir_v_72_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_72_before_2014"
dest_dir_v_81_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_81_before_2014"
dest_dir_v_82_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_82_before_2014"
dest_dir_v_83_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_83_before_2014"
dest_dir_v_84_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_84_before_2014"
dest_dir_v_85_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_85_before_2014"
dest_dir_v_86_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_86_before_2014"
dest_dir_v_87_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_87_before_2014"
dest_dir_v_88_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_88_before_2014"
dest_dir_v_89_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_89_before_2014"
dest_dir_v_90_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_90_before_2014"
dest_dir_v_91_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_91_before_2014"
dest_dir_v_92_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_92_before_2014"
dest_dir_v_97_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_97_before_2014"

dest_dir_v_TED_ESENDERS_before_2014 = "/home/jovyan/notebooks/falk/data/form_version/v_TED_ESENDERS_before_2014"




for fn in os.listdir(source_dir):
    tree = ET.parse(os.path.join(source_dir,fn))
    root = tree.getroot()
    #root_tag = root.tag
    #print(root_tag)
    #print(root[0][0].attrib)
    #root1 = tree.getroot([DOFFIN_ESENDERS])
    #print(root1)
    #test = root.get()
    #v = root[0][0].get('FORM')
    v_301 = root[0][1].get('FORM')
    #v_root_tag = root.tag
    #v_root_before_2014 = root.get('FORM')
    #print(v)
    #print(root)
    
     
    if v_301 == "F01":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F01, fn))
    elif v_301 == "F02":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F02, fn))
    elif v_301 == "F03":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F03, fn))
    elif v_301 == "F04":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F04, fn))
    elif v_301 == "F05":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F05, fn))
    elif v_301 == "F06":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F06, fn))
    elif v_301 == "F07":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F07, fn))
    elif v_301 == "F08":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F08, fn))
    elif v_301 == "F09":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F09, fn))
    elif v_301 == "F12":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F12, fn))
    elif v_301 == "F13":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F13, fn))
    elif v_301 == "F14":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F14, fn))
    elif v_301 == "F15":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F15, fn))
    elif v_301 == "F20":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F20, fn))
    elif v_301 == "F21":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F21, fn))
    elif v_301 == "F22":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F22, fn))
    elif v_301 == "F23":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F23, fn))
    elif v_301 == "F24":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F24, fn))
    elif v_301 == "F25":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F25, fn))
    elif v_301 == "F52":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F52, fn))
    elif v_301 == "F65":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F65, fn))
    elif v_301 == "F102":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_F102, fn))
    elif v_root_before_2014 == "1":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_1_before_2014, fn))
    elif v_root_before_2014 == "2":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_2_before_2014, fn))
    elif v_root_before_2014 == "3":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_3_before_2014, fn))
    elif v_root_before_2014 == "4":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_4_before_2014, fn))
    elif v_root_before_2014 == "5":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_5_before_2014, fn))
    elif v_root_before_2014 == "6":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_6_before_2014, fn))
    elif v_root_before_2014 == "7":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_7_before_2014, fn))
    elif v_root_before_2014 == "8":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_8_before_2014, fn))
    elif v_root_before_2014 == "9":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_9_before_2014, fn))
    elif v_root_before_2014 == "10":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_10_before_2014, fn))
    elif v_root_before_2014 == "11":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_10_before_2014, fn))
    elif v_root_before_2014 == "12":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_12_before_2014, fn))
    elif v_root_before_2014 == "13":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_13_before_2014, fn))
    elif v_root_before_2014 == "14":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_14_before_2014, fn))
    elif v_root_before_2014 == "51":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_51_before_2014, fn))
    elif v_root_before_2014 == "52":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_52_before_2014, fn))
    elif v_root_before_2014 == "53":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_53_before_2014, fn))
    elif v_root_before_2014 == "61":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_61_before_2014, fn))
    elif v_root_before_2014 == "62":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_62_before_2014, fn))
    elif v_root_before_2014 == "63":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_63_before_2014, fn))
    elif v_root_before_2014 == "64":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_64_before_2014, fn))
    elif v_root_before_2014 == "68":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_68_before_2014, fn))
    elif v_root_before_2014 == "69":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_69_before_2014, fn))
    elif v_root_before_2014 == "70":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_70_before_2014, fn))
    elif v_root_before_2014 == "72":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_72_before_2014, fn))
    elif v_root_before_2014 == "81":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_81_before_2014, fn))
    elif v_root_before_2014 == "82":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_82_before_2014, fn))
    elif v_root_before_2014 == "83":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_83_before_2014, fn))
    elif v_root_before_2014 == "84":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_84_before_2014, fn))
    elif v_root_before_2014 == "85":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_85_before_2014, fn))
    elif v_root_before_2014 == "86":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_86_before_2014, fn))
    elif v_root_before_2014 == "87":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_87_before_2014, fn))
    elif v_root_before_2014 == "88":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_88_before_2014, fn))
    elif v_root_before_2014 == "89":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_89_before_2014, fn))
    elif v_root_before_2014 == "90":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_90_before_2014, fn))
    elif v_root_before_2014 == "91":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_91_before_2014, fn))
    elif v_root_before_2014 == "92":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_92_before_2014, fn))
    elif v_root_before_2014 == "97":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_97_before_2014, fn))
    elif v_root_tag == "TED_ESENDERS":
        shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_v_TED_ESENDERS_before_2014, fn))
    
    else:
        #shutil.move(os.path.join(source_dir, fn), os.path.join(dest_dir_other, fn))
        pass

In [ ]:
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/1")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/10")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/101")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/102")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/103")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/11")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/12")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/13")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/14")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/15")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/16")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/17")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/18")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/2")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/3")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/4")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/5")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/6")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/7")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/8")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/9")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F01")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F02")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F03")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F04")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F05")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F06")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F07")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F08")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F09")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F102")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F12")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F13")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F14")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F15")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F20")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F21")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F22")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F23")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F24")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F25")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F52")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/F65")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/other")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_10_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_11_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_12_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_13_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_14_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_1_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_2_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_3_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_4_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_51_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_52_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_53_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_5_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_61_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_62_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_63_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_64_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_68_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_69_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_6_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_70_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_72_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_7_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_81_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_82_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_83_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_84_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_85_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_86_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_87_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_88_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_89_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_8_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_90_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_91_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_92_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_97_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_9_before_2014")
xml_to_json("/home/jovyan/notebooks/falk/data/form_version/v_TED_ESENDERS_before_2014")

In [ ]:
"""
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/1")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/10")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/101")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/102")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/103")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/11")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/12")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/13")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/14")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/15")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/16")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/17")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/18")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/2")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/3")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/4")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/5")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/6")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/7")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/8")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/9")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F01")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F02")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F03")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F04")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F05")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F06")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F07")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F08")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F09")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F102")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F12")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F13")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F14")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F15")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F20")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F21")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F22")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F23")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F24")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F25")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F52")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/F65")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/other")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_10_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_11_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_12_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_13_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_14_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_1_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_2_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_3_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_4_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_51_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_52_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_53_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_5_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_61_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_62_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_63_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_64_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_68_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_69_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_6_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_70_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_72_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_7_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_81_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_82_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_83_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_84_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_85_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_86_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_87_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_88_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_89_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_8_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_90_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_91_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_92_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_97_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_9_before_2014")
str_to_numb("/home/jovyan/notebooks/falk/data/form_version/v_TED_ESENDERS_before_2014")
"""

In [ ]:
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/1")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/10")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/101")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/102")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/103")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/11")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/12")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/13")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/14")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/15")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/16")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/17")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/18")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/2")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/3")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/4")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/5")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/6")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/7")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/8")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/9")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F01")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F02")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F03")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F04")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F05")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F06")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F07")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F08")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F09")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F102")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F12")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F13")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F14")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F15")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F20")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F21")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F22")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F23")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F24")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F25")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F52")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/F65")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/other")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_10_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_11_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_12_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_13_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_14_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_1_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_2_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_3_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_4_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_51_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_52_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_53_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_5_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_61_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_62_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_63_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_64_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_68_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_69_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_6_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_70_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_72_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_7_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_81_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_82_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_83_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_84_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_85_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_86_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_87_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_88_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_89_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_8_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_90_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_91_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_92_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_97_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_9_before_2014")
json_strip_list2("/home/jovyan/notebooks/falk/data/form_version/v_TED_ESENDERS_before_2014")

|Form |From year | To year|
| :- | :- | :- |
|1 | 2014 | 2016 |
|2 | 2014 | 2016
|3 | 2014 | 2016
|4 | 2014 | 2016
|5 | 2014 | 2016
|6 | 2014 | 2016
|7 | 2014 | 2016
|8 | 2014 | 2016
|9 | 2014 | 2016
|10 | 2014 | 2016
|11 | 2014 | 2016
|12 | 2014 | 2016
|13 | 2014 | 2016
|14 | 2014 | 2022
|15 | 2014 | 2016
|16 | 2014 | 2021
|17 | 2014 | 2022
|18 | 2014 | 2022
|101 | 2014 | 2016
|102 | 2014 | 2016
|103 | 2014 | 2016

In [ ]:
# Null and all columns with only null will be deleted.

json_to_norm("/home/jovyan/notebooks/falk/data/form_version/1", '/home/jovyan/notebooks/falk/1.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/10", '/home/jovyan/notebooks/falk/10.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/101", '/home/jovyan/notebooks/falk/101.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/102", '/home/jovyan/notebooks/falk/102.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/103", '/home/jovyan/notebooks/falk/103.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/11", '/home/jovyan/notebooks/falk/11.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/12", '/home/jovyan/notebooks/falk/12.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/13", '/home/jovyan/notebooks/falk/13.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/15", '/home/jovyan/notebooks/falk/15.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/2", '/home/jovyan/notebooks/falk/2.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/3", '/home/jovyan/notebooks/falk/3.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/4", '/home/jovyan/notebooks/falk/4.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/5", '/home/jovyan/notebooks/falk/5.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/6", '/home/jovyan/notebooks/falk/6.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/7", '/home/jovyan/notebooks/falk/7.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/8", '/home/jovyan/notebooks/falk/8.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/9", '/home/jovyan/notebooks/falk/9.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/other", '/home/jovyan/notebooks/falk/other.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_10_before_2014", '/home/jovyan/notebooks/falk/v_10_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_11_before_2014", '/home/jovyan/notebooks/falk/v_11_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_12_before_2014", '/home/jovyan/notebooks/falk/v_12_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_13_before_2014", '/home/jovyan/notebooks/falk/v_13_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_14_before_2014", '/home/jovyan/notebooks/falk/v_14_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_1_before_2014", '/home/jovyan/notebooks/falk/v_1_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_2_before_2014", '/home/jovyan/notebooks/falk/v_2_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_3_before_2014", '/home/jovyan/notebooks/falk/v_3_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_4_before_2014", '/home/jovyan/notebooks/falk/v_4_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_51_before_2014", '/home/jovyan/notebooks/falk/v_51_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_52_before_2014", '/home/jovyan/notebooks/falk/v_52_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_53_before_2014", '/home/jovyan/notebooks/falk/v_53_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_5_before_2014", '/home/jovyan/notebooks/falk/v_5_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_61_before_2014", '/home/jovyan/notebooks/falk/v_61_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_62_before_2014", '/home/jovyan/notebooks/falk/v_62_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_63_before_2014", '/home/jovyan/notebooks/falk/v_63_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_64_before_2014", '/home/jovyan/notebooks/falk/v_64_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_68_before_2014", '/home/jovyan/notebooks/falk/v_68_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_69_before_2014", '/home/jovyan/notebooks/falk/v_69_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_6_before_2014", '/home/jovyan/notebooks/falk/v_6_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_70_before_2014", '/home/jovyan/notebooks/falk/v_70_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_72_before_2014", '/home/jovyan/notebooks/falk/v_72_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_7_before_2014", '/home/jovyan/notebooks/falk/v_7_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_81_before_2014", '/home/jovyan/notebooks/falk/v_81_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_82_before_2014", '/home/jovyan/notebooks/falk/v_82_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_83_before_2014", '/home/jovyan/notebooks/falk/v_83_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_84_before_2014", '/home/jovyan/notebooks/falk/v_84_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_85_before_2014", '/home/jovyan/notebooks/falk/v_85_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_86_before_2014", '/home/jovyan/notebooks/falk/v_86_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_87_before_2014", '/home/jovyan/notebooks/falk/v_87_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_88_before_2014", '/home/jovyan/notebooks/falk/v_88_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_89_before_2014", '/home/jovyan/notebooks/falk/v_89_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_8_before_2014", '/home/jovyan/notebooks/falk/v_8_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_90_before_2014", '/home/jovyan/notebooks/falk/v_90_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_91_before_2014", '/home/jovyan/notebooks/falk/v_91_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_92_before_2014", '/home/jovyan/notebooks/falk/v_92_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_97_before_2014", '/home/jovyan/notebooks/falk/v_97_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_9_before_2014", '/home/jovyan/notebooks/falk/v_9_before_2014.json')
json_to_norm("/home/jovyan/notebooks/falk/data/form_version/v_TED_ESENDERS_before_2014", '/home/jovyan/notebooks/falk/v_TED_ESENDERS_before_2014.json')

In [ ]:
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/16", '/home/jovyan/notebooks/falk/16.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/17", '/home/jovyan/notebooks/falk/17.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/18", '/home/jovyan/notebooks/falk/18.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/14", '/home/jovyan/notebooks/falk/14.json')

json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F01", '/home/jovyan/notebooks/falk/F01.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F02", '/home/jovyan/notebooks/falk/F02.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F03", '/home/jovyan/notebooks/falk/F03.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F04", '/home/jovyan/notebooks/falk/F04.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F05", '/home/jovyan/notebooks/falk/F05.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F06", '/home/jovyan/notebooks/falk/F06.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F07", '/home/jovyan/notebooks/falk/F07.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F08", '/home/jovyan/notebooks/falk/F08.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F09", '/home/jovyan/notebooks/falk/F09.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F102", '/home/jovyan/notebooks/falk/F102.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F12", '/home/jovyan/notebooks/falk/F12.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F13", '/home/jovyan/notebooks/falk/F13.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F14", '/home/jovyan/notebooks/falk/F14.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F15", '/home/jovyan/notebooks/falk/F15.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F20", '/home/jovyan/notebooks/falk/F20.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F21", '/home/jovyan/notebooks/falk/F21.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F22", '/home/jovyan/notebooks/falk/F22.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F23", '/home/jovyan/notebooks/falk/F23.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F24", '/home/jovyan/notebooks/falk/F24.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F25", '/home/jovyan/notebooks/falk/F25.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F52", '/home/jovyan/notebooks/falk/F52.json')
json_to_norm_with_null("/home/jovyan/notebooks/falk/data/form_version/F65", '/home/jovyan/notebooks/falk/F65.json')

In [14]:
spark.stop()